# WK - Projekt 2

### Adam Wiatrowski, 148510
### Maciej Wieczorek, 148141

In [ ]:
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
from IPython.display import display, clear_output

In [ ]:
def imshow(a):
  a = a.clip(0, 255).astype('uint8')
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display(Image.fromarray(a))

In [ ]:
def imshow_pixel(p):
    imshow(np.full((64, 64, 3), p))

In [ ]:
path = "part_3.mp4"

# wczytanie klatki
cap = cv2.VideoCapture(path)
ret, frame = cap.read()

# maska stołu
points = np.array([[[1600, 860], [327, 860], [533, 125], [1350, 125]]], np.int32)
mask = np.zeros(frame.shape[:2], dtype="uint8")
cv2.fillPoly(mask, points, 1)
masked = cv2.bitwise_and(frame, frame, mask=mask)


frame_hsv = cv2.cvtColor(masked, cv2.COLOR_BGR2HSV)
frame_gray = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)

imshow(np.concatenate([masked, frame_hsv], 1))

In [ ]:
def find_circles(img):

  bluerd = cv2.medianBlur(img, 9)

  minDist = 5
  param1 = 30 # first method-specific parameter. In case of HOUGH_GRADIENT, it is the higher threshold of the two passed to the Canny edge detector (the lower one is twice smaller).
  param2 =  25 # im mniejszy, tym wiecej fake
  minRadius = 10
  maxRadius = 25

  circles = cv2.HoughCircles(bluerd, cv2.HOUGH_GRADIENT, 1, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)

  mask = np.zeros(img.shape[:2])

  if circles is not None:
      circles = np.uint16(np.around(circles))
      for i in circles[0,:]:
          
          # wziete z neta
          center = (i[0], i[1])
          radius = i[2]

          x, y = int(center[0] - radius), int(center[1] - radius)
          width, height = int(radius * 2), int(radius * 2)
          if x >= 0 and y >= 0 and (x + width) < img.shape[1] and (y + height) < img.shape[0]:
            roi = img[y:y+height, x:x+width]

            if np.mean(roi) < 128:
              cv2.circle(img, center, radius, (0, 0, 255), 2)
              cv2.circle(mask, center, radius, (255), thickness=-1)


  imshow(mask)
  return mask

In [ ]:
frame_copy = frame_gray.copy()

black_mask = find_circles(frame_copy)

In [ ]:
# z hsv wez maske 
lower = np.array([50, 70, 120])
upper = np.array([70, 255, 240])


new_mask = cv2.inRange(frame_hsv, lower, upper)

reversed_mask = cv2.bitwise_not(new_mask)

test = cv2.bitwise_and(masked, masked, mask=reversed_mask)

imshow(test)
#lower = np.array([50, 150, 150])
#upper = np.array([70, 255, 240])

In [ ]:
test_gray = cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
find_circles(test_gray)

In [ ]:
def find_counturs(img):
    #kernel = np.ones((5,5),np.uint8)
    #morp = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)


    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)



    contour_img = np.zeros_like(img)

    cv2.drawContours(contour_img, contours, -1, (255, 255, 255), 3)

    imshow(contour_img)

In [ ]:
channel_h = frame_hsv[:, :, 0].copy()
np.histogram(channel_h)

In [ ]:
points = np.array([[[1600, 860], [327, 860], [533, 125], [1350, 125]]], np.int32)

mask2 = np.zeros(frame.shape[:2], dtype="uint8")
cv2.fillPoly(mask2, points, 1)
imshow(mask2*255)

In [ ]:
channel_h = frame_hsv[:, :, 0].copy()
channel_h[(channel_h < 45) | (channel_h > 67)] = 255

#imshow(channel_h)


combined_mask = np.where((channel_h == 255) | (black_mask == 255), 255, 0).astype(np.uint8)
combined_mask = np.where((combined_mask == 255) & (mask == 1), 255, 0).astype(np.uint8)


imshow(np.concatenate([combined_mask], 1))
# tam gdzie sa biale elementy w jednym lub drugim (channel_h i black_mask)
#find_counturs(channel_h)

In [ ]:
# MASKA

In [ ]:
def draw_rectangles(ctrs, img):
    
    output = img.copy()
    
    for i in range(len(ctrs)):
    
        M = cv2.moments(ctrs[i]) # moments
        rot_rect = cv2.minAreaRect(ctrs[i])
        w = rot_rect[1][0] # width
        h = rot_rect[1][1] # height

        area = w*h
        
        if area < 1200 and area > 300:
            box = np.int64(cv2.boxPoints(rot_rect))
            cv2.drawContours(output,[box],0,(255,100,0),2) # draws box
        
    return output

In [ ]:
def filter_contours(contours):
    c = []
    for cnt in contours:
        cnt_area = cv2.contourArea(cnt)
        if cnt_area < 170 or cnt_area > 1000:
            continue
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        if abs(hull_area-cnt_area) > 50:
            continue

        c.append(hull)
    return tuple(c)

In [ ]:
imshow(combined_mask)

In [ ]:
combined_mask_edges = cv2.Canny(combined_mask, 100, 200)

In [ ]:
imshow(combined_mask_edges)

In [ ]:
ctrs, hierarchy = cv2.findContours(combined_mask_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
ctrs = filter_contours(ctrs)

# filtered_img = np.zeros_like(combined_mask)
    
# Loop through all contours and add only those whose area is less than the max_area
# etected_objects = draw_rectangles(ctrs, filtered_img)
cv2.drawContours(masked, ctrs, -1, (0, 0, 255), 2)
imshow(masked)

In [25]:
import cv2
import numpy as np
path = 'part_2.mp4'

def filter_contours(contours):
    c = []
    for cnt in contours:
        cnt_area = cv2.contourArea(cnt)
        if cnt_area < 170 or cnt_area > 1000:
            continue
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        if abs(hull_area-cnt_area) > 50:
            continue

        c.append(hull)
    return tuple(c)

def find_black(img):

  bluerd = cv2.medianBlur(img, 9)

  minDist = 5
  param1 = 30 # first method-specific parameter. In case of HOUGH_GRADIENT, it is the higher threshold of the two passed to the Canny edge detector (the lower one is twice smaller).
  param2 =  25 # im mniejszy, tym wiecej fake
  minRadius = 10
  maxRadius = 25

  circles = cv2.HoughCircles(bluerd, cv2.HOUGH_GRADIENT, 1, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)

  mask = np.zeros(img.shape[:2])

  if circles is not None:
      circles = np.uint16(np.around(circles))
      for i in circles[0,:]:
          
          # wziete z neta
          center = (i[0], i[1])
          radius = i[2]

          x, y = int(center[0] - radius), int(center[1] - radius)
          width, height = int(radius * 2), int(radius * 2)
          if x >= 0 and y >= 0 and (x + width) < img.shape[1] and (y + height) < img.shape[0]:
            roi = img[y:y+height, x:x+width]

            if np.mean(roi) < 128:
              cv2.circle(img, center, radius, (0, 0, 255), 2)
              cv2.circle(mask, center, radius, (255), thickness=-1)


  return mask

def process_frame(frame):

    points = np.array([[[1600, 860], [327, 860], [533, 125], [1350, 125]]], np.int32)
    mask = np.zeros(frame.shape[:2], dtype="uint8")
    cv2.fillPoly(mask, points, 1)
    masked = cv2.bitwise_and(frame, frame, mask=mask)


    frame_hsv = cv2.cvtColor(masked, cv2.COLOR_BGR2HSV)
    frame_gray = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)

    black_mask = find_black(frame_gray)

    channel_h = frame_hsv[:, :, 0]
    channel_h[(channel_h < 45) | (channel_h > 67)] = 255

    combined_mask = np.where((channel_h == 255) | (black_mask == 255), 255, 0).astype(np.uint8)
    combined_mask = np.where((combined_mask == 255) & (mask == 1), 255, 0).astype(np.uint8)

    combined_mask_edges = cv2.Canny(combined_mask, 100, 200)


    ctrs, _ = cv2.findContours(combined_mask_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    ctrs = filter_contours(ctrs)
    frame_cnts = frame.copy()
    cv2.drawContours(frame_cnts, ctrs, -1, (0, 0, 255), 2)

    return frame_cnts

# wczytanie klatki
cap = cv2.VideoCapture(path)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    if cv2.waitKey(1) == ord('q'):
        break

    cv2.imshow('frame', process_frame(frame))

cap.release()
cv2.destroyAllWindows()

In [34]:
import cv2
import numpy as np
path = 'part_2.mp4'

def filter_contours(contours):
    c = []
    for cnt in contours:
        cnt_area = cv2.contourArea(cnt)
        if cnt_area < 170 or cnt_area > 1000:
            continue
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        if abs(hull_area-cnt_area) > 50:
            continue

        c.append(hull)
    return tuple(c)

def find_black(img):

  bluerd = cv2.medianBlur(img, 9)

  minDist = 5
  param1 = 30 # first method-specific parameter. In case of HOUGH_GRADIENT, it is the higher threshold of the two passed to the Canny edge detector (the lower one is twice smaller).
  param2 =  25 # im mniejszy, tym wiecej fake
  minRadius = 10
  maxRadius = 25

  circles = cv2.HoughCircles(bluerd, cv2.HOUGH_GRADIENT, 1, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)

  mask = np.zeros(img.shape[:2], dtype=np.uint8)

  if circles is not None:
      circles = np.uint16(np.around(circles))
      for i in circles[0,:]:
          
          # wziete z neta
          center = (i[0], i[1])
          radius = i[2]

          x, y = int(center[0] - radius), int(center[1] - radius)
          width, height = int(radius * 2), int(radius * 2)
          if x >= 0 and y >= 0 and (x + width) < img.shape[1] and (y + height) < img.shape[0]:
            roi = img[y:y+height, x:x+width]

            if np.mean(roi) < 128:
              cv2.circle(mask, center, radius, (1), thickness=-1)


  return mask

def process_frame(frame):

    points = np.array([[[1600, 860], [327, 860], [533, 125], [1350, 125]]], np.int32)
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, points, 1)
    masked = cv2.bitwise_and(frame, frame, mask=mask)


    frame_hsv = cv2.cvtColor(masked, cv2.COLOR_BGR2HSV)
    frame_gray = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)

    black_mask = find_black(frame_gray)

    channel_h = frame_hsv[:, :, 0]
    channel_h_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    channel_h_mask[(channel_h < 45) | (channel_h > 67)] = 1

    masks = cv2.bitwise_or(black_mask, channel_h_mask)

    combined_mask = np.where(masks == 1, 255, 0).astype(np.uint8)

    combined_mask_edges = cv2.Canny(combined_mask, 100, 200)


    ctrs, _ = cv2.findContours(combined_mask_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    ctrs = filter_contours(ctrs)
    frame_cnts = frame.copy()
    cv2.drawContours(frame_cnts, ctrs, -1, (0, 0, 255), 2)

    return frame_cnts

cap = cv2.VideoCapture(path)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    if cv2.waitKey(1) == ord('q'):
        break

    cv2.imshow('frame', process_frame(frame))

cap.release()
cv2.destroyAllWindows()